In [65]:
import pandas as pd
from IPython.display import display, HTML
import warnings

# Define function that will be reused to prepare 3 csv files covering years 2000-23

def clean_df(df, years, old_year_names, keep_columns):
    """
    Transforms the given DataFrame by keeping selected columns, renaming them, padding values with leading zeros,
    creating a FIPS column, and melting the DataFrame to a long format.
    
    Parameters:
    - df: The original DataFrame.
    - years: List of new year column names.
    - old_year_names: List of old year column names to be renamed.
    - keep_columns: List of columns to keep initially.
    
    Returns:
    - Transformed DataFrame in long format with columns FIPS, Year, and POP.
    """
    # Keep selected columns
    keep_columns_all = keep_columns + old_year_names
    df = df[keep_columns_all]

    # Rename columns
    new_names = keep_columns + years
    df.columns = new_names

    # Pad the values of specific columns with leading zeros and create FIPS column
    df.loc[:,['STATE']] = df['STATE'].astype(str).str.zfill(2)
    df.loc[:,['COUNTY']] = df['COUNTY'].astype(str).str.zfill(3)
    df.loc[:,['FIPS']] = df['STATE'] + df['COUNTY']

    # Select relevant columns
    selected_columns = ['FIPS'] + years
    df = df[selected_columns]

    # Use the melt function to transform the DataFrame
    df = pd.melt(df, id_vars=['FIPS'], var_name='Year', value_name='POP')

    return df

In [168]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #Prepare 2020-23 Data
    #####################
    
    # Load the CSV file
    df = pd.read_csv('co_2020-23.csv',engine='python',encoding='latin1')
    
    # Parameters
    years = list(range(2020,2024)) 
    years = [str(item) for item in years]
    old_year_names = ['ESTIMATESBASE2020','POPESTIMATE2021', 'POPESTIMATE2022', 'POPESTIMATE2023']
    keep_columns = ['STATE', 'COUNTY'] 
    
    df1 = clean_df(df, years, old_year_names, keep_columns)
    
    # Display the DataFrame with HTML formatting
    display(HTML(df1.head().to_html()))

,FIPS,Year,POP
0,01000,2020,5024294
1,01001,2020,58809
2,01003,2020,231768
3,01005,2020,25229
4,01007,2020,22301


In [169]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #Prepare 2010-19 Data
    #####################
    
    # Load the CSV file
    df = pd.read_csv('co_2010-19.csv',engine='python',encoding='latin1')
    
    # Parameters
    years = list(range(2010,2020)) 
    years = [str(item) for item in years]
    old_year_names = ['CENSUS2010POP','POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014'
                   , 'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017', 'POPESTIMATE2018', 'POPESTIMATE2019']
    keep_columns = ['STATE', 'COUNTY'] 
    
    df2 = clean_df(df, years, old_year_names, keep_columns)
    
    # Display the DataFrame with HTML formatting
    display(HTML(df2.head().to_html()))

,FIPS,Year,POP
0,01000,2010,4779736
1,01001,2010,54571
2,01003,2010,182265
3,01005,2010,27457
4,01007,2010,22915


In [170]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #Prepare 2000-09 Data
    #####################
    
    # Load the CSV file
    df = pd.read_csv('co_2000-09.csv',engine='python',encoding='latin1')
    
    # Parameters
    years = list(range(2000,2010)) 
    years = [str(item) for item in years]
    old_year_names = ['ESTIMATESBASE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003',
       'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006','POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009']
    keep_columns = ['STATE', 'COUNTY'] 
    
    df3 = clean_df(df, years, old_year_names, keep_columns)
    
    # Display the DataFrame with HTML formatting
    display(HTML(df3.head().to_html()))

,FIPS,Year,POP
0,01000,2000,4447207
1,01001,2000,43751
2,01003,2000,140416
3,01005,2000,29042
4,01007,2000,19856


In [132]:
#Prepare 1980-89 Data
#####################

# Read the Excel file with all sheets
xls = pd.ExcelFile('co-1980-89_tabs.xls')

# Dictionary to hold cleaned DataFrames
cleaned_dfs = {}

# Iterate through each sheet
for sheet_name in xls.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    
    # Drop rows 1-5 and 7 (note: DataFrame index starts at 0)
    df = df.drop([0, 1, 2, 3, 4, 6])
    
    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # Set the first row as the header
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    
    # Add the cleaned DataFrame to the dictionary
    cleaned_dfs[sheet_name] = df

# Concatenate all DataFrames in the dictionary
df0 = pd.concat(cleaned_dfs.values(), ignore_index=True)

In [150]:
#Prepare 1980-89 Data contd.
#####################

df4 = df0.copy()

# Sum all ages
df4['POP_all_ages'] = df4.iloc[:,3:].sum(axis=1)

# Keep selected columns
df4 = df4[['Year of Estimate', 'FIPS State and County Codes',
       'Race/Sex Indicator', 'POP_all_ages']]

# Rename columns
df4.columns = ['Year', 'FIPS',
       'Race/Sex Indicator', 'POP_all_ages']

# Groupby sum 'Race/Sex Indicator' by FIPS and Year
df4 = df4.groupby(['FIPS','Year'], as_index=False).agg(POP=('POP_all_ages', 'sum'))

df4.head()

,FIPS,Year,POP
0,01001,1980,32217
1,01001,1981,31985
2,01001,1982,32038
3,01001,1983,32057
4,01001,1984,32130


In [177]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    #Prepare 1990-99 Data
    #####################
    
    years = list(range(1990,2000)) 
    years = [str(item) for item in years]
    
    # Specify the column names
    column_names = ['Year', 'FIPS', 'Age Group', 'Race-Sex', 'Ethnic origin', 'POP']
    
    # Create an empty DataFrame with the specified column names
    df = pd.DataFrame(columns=column_names)
    
    # Define the column widths and names
    column_widths = [2, 7, 3, 2, 2, 7]  
    
    # Dictionary to hold cleaned DataFrames
    cleaned_dfs = {}
    
    # Read the .txt files
    for i in years:
        file_name = f'stch-icen{i}.txt'
        df = pd.read_fwf(file_name, widths=column_widths, names=column_names)
    
        # Groupby sum 'Race/Sex Indicator' by FIPS and Year
        df = df.groupby(['FIPS','Year'], as_index=False).agg(POP=('POP', 'sum'))
    
        df.loc[:,['Year']] = i
    
        # Add the cleaned DataFrame to the dictionary
        cleaned_dfs[i] = df

In [179]:
#Prepare 1990-99 Data contd.
#####################

# Concatenate all DataFrames in the dictionary
df5 = pd.concat(cleaned_dfs.values(), ignore_index=True)

# Print the DataFrame to verify
display(HTML(df5.head().to_html()))
display(HTML(df5.tail().to_html()))

,FIPS,Year,POP
0,1001,1990,34353
1,1003,1990,98955
2,1005,1990,25505
3,1007,1990,16697
4,1009,1990,39473


,FIPS,Year,POP
31405,56037,1999,38142
31406,56039,1999,17672
31407,56041,1999,19901
31408,56043,1999,8425
31409,56045,1999,6667


In [183]:
# Concatenate all period dfs
df_out = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)

display(HTML(df_out.head().to_html()))
display(HTML(df_out.tail().to_html()))

# Export the DataFrame to a CSV file
df_out.to_excel('US County Pop Estimates 1980-2023.xlsx', index=False)

,FIPS,Year,POP
0,01000,2020,5024294
1,01001,2020,58809
2,01003,2020,231768
3,01005,2020,25229
4,01007,2020,22301


,FIPS,Year,POP
139465,56037,1999,38142
139466,56039,1999,17672
139467,56041,1999,19901
139468,56043,1999,8425
139469,56045,1999,6667
